The seismic modes are stored in various files. I'm going to put them all together in a single file.

In [1]:
import numpy as np
import pandas as pd
import glob
from astropy.io import ascii

## Add LEGACY modes
The LEGACY modes are more accessible in a single file, so lets use that as the base.

In [2]:
copper = ascii.read('legacy_modeids.txt').to_pandas()

Add the source labels to the array

In [3]:
copper['source'] = ['L' for x in range(len(copper))]
copper.head()

,KIC,n,l,Freq,e_Freq,E_Freq,Amp,e_Amp,E_Amp,Width,e_Width,E_Width,l_lnK,lnK,source
0,1435467,11,1,889.25,0.47,0.53,NaN,NaN,NaN,NaN,NaN,NaN,>,6.00,L
1,1435467,11,2,920.04,1.82,1.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.43,L
2,1435467,12,0,928.22,0.94,0.71,2.39,0.14,0.13,4.79,0.57,0.96,NaN,2.11,L
3,1435467,12,1,957.04,0.50,0.46,NaN,NaN,NaN,NaN,NaN,NaN,>,6.00,L
4,1435467,12,2,989.14,1.24,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.63,L


## Add Kages modes

Some stars are duplicated between LEGACY and Kages. In these cases, we have kept the stellar parameters reported in LEGACY, so we want to make sure we use the modes from legacy as well.

We will read in our list of target stars and read in modes for Kages stars in that list.

In [4]:
malatium = pd.read_csv('malatium.csv',index_col=0)
kics = malatium[malatium.source == 'K'].KIC.values

In [5]:
for kic in kics:
    sfile = glob.glob('kages_modeids/'+str(kic)+'/*/*.fre',recursive=True)[0]
    modes = pd.read_csv(sfile, names=['n','l','Freq','e_Freq'], sep='\s+')
    modes['KIC'] = [kic for x in range(len(modes))]
    modes['source'] = ['K' for x in range(len(modes))]
    copper = pd.concat([copper, modes],sort=False, ignore_index=True).reset_index(drop=True)

In [8]:
copper.head()

,KIC,n,l,Freq,e_Freq,E_Freq,Amp,e_Amp,E_Amp,Width,e_Width,E_Width,l_lnK,lnK,source
0,1435467,11,1,889.25,0.47,0.53,NaN,NaN,NaN,NaN,NaN,NaN,>,6.00,L
1,1435467,11,2,920.04,1.82,1.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.43,L
2,1435467,12,0,928.22,0.94,0.71,2.39,0.14,0.13,4.79,0.57,0.96,NaN,2.11,L
3,1435467,12,1,957.04,0.50,0.46,NaN,NaN,NaN,NaN,NaN,NaN,>,6.00,L
4,1435467,12,2,989.14,1.24,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.63,L


## Now lets save the output

In [9]:
copper.to_csv('copper.csv')